In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
from skinet.utils.athena_tools import *
warnings.filterwarnings("ignore")

In [ ]:
# matplotlib 한글 나오게 하기

import matplotlib as mpl
import matplotlib.pylab as plt2 

plt2.rcParams["font.family"] = "NanumGothincCoding"
plt2.rcParams["font.size"] = 12
plt2.rcParams["figure.figsize"] = (14, 4) 
mpl.rcParams['axes.unicode_minus'] = False 

In [ ]:
ym = '202010' # 전월 
fm = '202009' # 최초 청구월 기준
ft = 20200801 # 최종 통화일 기준

In [ ]:
# tm list header update 
conn = athena_connection()
drop_table_in_athena(conn, 'default.tm_list_header')
temp = load_from_athena(conn, '''
SELECT DISTINCT listid
     , listname
     , listtypecd
     , extracteddate
     , extractedtime
     , extractorid
     , requestcnt
     , extractedcnt 
     , isallocation
     , teamcd
     , extractedgbn
FROM telepro.tm_list_cat
WHERE listtypecd = '35'
  AND extractorid = 'K1170127'
''')
load_to_athena(conn, temp, 'default.tm_list_header')
del temp

In [ ]:
# # 보험 가입자 리스트 추출

# conn = athena_connection()
# ins = load_from_athena(conn, '''
# SELECT DISTINCT b.ctnum
#      , b.rqnum
#      , b.aedat
#      , b.ins_type
#      , b.ins_amt
#      , b.ins_amt_type
#      , b.ins_payment
#      , b.ins_stat
#      , b.ins_user4
#      , c.ins_stat_nm
#      , c.ins_amt_type_nm
#      , d.ctnum_old
#   FROM (SELECT DISTINCT ctnum, ins_type, ins_amt, ins_amt_type, ins_payment, ins_stat, rqnum, aedat, ins_user4
#              FROM sap.zsdt4150m
#             WHERE ins_type IN ('02', '03', '07')) b
# LEFT JOIN (SELECT DISTINCT ctnum, rqnum, ins_stat_nm, ins_amt_type_nm FROM mart.zsdr6051) c
#        ON b.ctnum = c.ctnum
#       AND b.rqnum = c.rqnum
# LEFT JOIN (SELECT DISTINCT ctnum, ctnum_old FROM mart.zsdr6001 WHERE ctnum_old <> '') d
#        ON b.ctnum = d.ctnum
# ''') 

In [ ]:
conn = athena_connection()
magam = load_from_athena(conn, '''

SELECT a.kcalss
	 , d.kunnr
	 , d.ctnum
     , a.conn_dept
     , a.conn_emp
     , a.cost_dept
     , a.cost_emp
     , a.ctnum as ctnum_hash
     , a.cstat
     , a.gad_name
     , a.gad_start
     , a.conn_start
     , a.conn_end
     , CAST(a.monexp_amt AS DOUBLE) AS monexp_amt
     , a.conn_type
     , a.ctype_l
     , a.ctype_m
     , a.ctype_s
     , a.intro_asp
     , a.itype_l
     , a.itype_m
     , a.itype_s
     , a.itype_sdesc
     , a.kunnr as kunnr_hash
     , c.name1
     , a.gad_add01
     , a.post_add01
     , a.first_bildate
     , c.telf2
--     , (SELECT ETAX_EMAIL FROM SAP.ZSDT4021 WHERE CTNUM = A.CTNUM AND RQNUM = A.RQNUM AND AMT_TYPE = 'F3' AND MANDT = '100') AS 전자계산서EMAIL
--     , (SELECT GROS_CODE FROM SAP.ZSDT1110 WHERE KUNNR = A.KUNNR AND MANDT = '100' AND BUKRS = '1000') AS GROSCODE
     , a.svc_str
     , a.pay_type
     , a.depo_type
     , a.line_type
     , a.pck_str
--     , (SELECT BIR_DATE FROM SAP.ZSDT1110 WHERE KUNNR = A.KUNNR AND MANDT = '100' AND BUKRS = '1000') AS 생년월일
     , SUBSTR(c.stcd1, 1 ,6) AS B_YMD
     , SUBSTR(c.stcd1, 7 ,1) AS SEX
     , c.stcd2
     , c.j_1kftbus AS uptae
     , c.j_1kftind AS jongmok
     , a.gros_code
     , a.gross_type
     , a.kvgr4
     , a.gad_pstlz
     , a.gad_add02
     , a.post_add02
     , b.result1
     , b.result2
     , b.result3
     , b.result4
     , b.result5
     , b.result6
     , b.sms_yn
     , b.email_yn
     , b.tm_yn
     , b.dm_yn
     , b.sk1_yn
     , b.sk2_yn
     , b.adt1_yn
     , b.aedat
     , b.aenam
     , e.ctrt_id
FROM sap.zsdt4013      AS a 
LEFT JOIN (SELECT ctnum, kunnr, sms_yn, email_yn, tm_yn, dm_yn, sk1_yn, sk2_yn, adt1_yn, aedat, aenam, result1, result2, result3, result4, result5, result6 FROM sap.zsdt3748 WHERE usest = 'X') AS b
      ON a.ctnum = b.ctnum
LEFT JOIN (SELECT kunnr, stcd2, stcd1, telf2, name1, j_1kftbus, j_1kftind FROM sap.kna1 WHERE mandt = '100') AS c
      ON a.kunnr = c.kunnr
LEFT JOIN (select distinct ctnum_hash, ctnum, kunnr_hash, kunnr from adt_work.hash_info) d
       ON a.ctnum = d.ctnum_hash
LEFT JOIN (select distinct ctrt_id, ctnum from telepro.tb_nsok_mapping) e
       ON a.ctnum = e.ctnum
  WHERE a.mandt = '100' 
    AND a.stmon = \'{}\'  --년월 변경
    AND a.bukrs = '1000'
    AND a.cstat IN ('1','2');

'''.format(ym)) 

In [ ]:
from skinet.utils.athena_tools import *
conn = athena_connection()
tm = load_from_athena(conn, '''
WITH t1 AS (
SELECT b.kunnr_hash AS kunnr, MAX(a.talkenddate) AS talkenddate
  FROM telepro.tb_contact a
LEFT JOIN (SELECT ctnum_hash, kunnr_hash FROM adt_work.hash_info) b
       ON a.s25csq = b.ctnum_hash
WHERE a.listid in (SELECT DISTINCT listid FROM default.tm_list_header)
GROUP BY b.kunnr_hash
)

, t2 AS (
SELECT b.kunnr_hash AS kunnr, a.talkenddate, MAX(a.contactendtime) AS contactendtime
  FROM telepro.tb_contact a
LEFT JOIN (SELECT ctnum_hash, kunnr_hash FROM adt_work.hash_info) b
       ON a.s25csq = b.ctnum_hash
WHERE a.listid in (SELECT DISTINCT listid FROM default.tm_list_header)
GROUP BY b.kunnr_hash, a.talkenddate
)

, t3 AS (
SELECT b.kunnr_hash AS kunnr, a.talkenddate, a.contactendtime, a.dialresultcd
  FROM telepro.tb_contact a
LEFT JOIN (SELECT ctnum_hash, kunnr_hash FROM adt_work.hash_info) b
       ON a.s25csq = b.ctnum_hash
 WHERE a.listid in (SELECT DISTINCT listid FROM default.tm_list_header)
)

, t4 AS (
SELECT b.kunnr_hash AS kunnr, COUNT(DISTINCT a.listid) AS contact_cnt
  FROM telepro.tb_contact a
LEFT JOIN (SELECT ctnum_hash, kunnr_hash FROM adt_work.hash_info) b
       ON a.s25csq = b.ctnum_hash
 WHERE a.listid in (SELECT DISTINCT listid FROM default.tm_list_header) 
GROUP BY b.kunnr_hash
)

SELECT DISTINCT t1.kunnr, t1.talkenddate, t2.contactendtime, t3.dialresultcd, t4.contact_cnt
  FROM t1
LEFT JOIN t2 
       ON t1.kunnr = t2.kunnr
      AND t1.talkenddate = t2.talkenddate
LEFT JOIN t3
       ON t1.kunnr = t3.kunnr
      AND t1.talkenddate = t3.talkenddate
      AND t2.contactendtime = t3.contactendtime
LEFT JOIN t4
       ON t1.kunnr = t4.kunnr
''') 

In [ ]:
magam.shape

In [ ]:
tm_drop_duplicates = tm.drop_duplicates()
tm_drop_duplicates.shape

In [ ]:
magam = pd.merge(magam, tm, how='left', left_on='kunnr_hash', right_on='kunnr')

In [ ]:
magam = magam.drop_duplicates()
magam.shape

In [ ]:
drop_col = ['kcalss', 'conn_dept', 'conn_emp', 'cost_dept', 'cost_emp', 'intro_asp', 'itype_sdesc', 'svc_str', 'pay_type', 'depo_type',
            'result1', 'result2', 'result3', 'result4', 'result5', 'result6']
magam.drop(drop_col, axis=1, inplace=True)

In [ ]:
magam_copy = magam.copy()

In [ ]:
# magam = magam_copy.copy()

In [ ]:
cond1 = magam['cstat'].isin(['1'])
cond2 = magam['conn_type'].isin(['A02'])                                                    # CMS, Non CMS

In [ ]:
temp1 = magam[cond1 & cond2]

In [ ]:
temp1['telf2'] = temp1['telf2'].str.replace('-|/|' '|'  '', '', regex=True)
temp1['telf2'] = np.where(temp1['telf2'].str[0] != '0', "0"+temp1['telf2'], temp1['telf2'])
temp1['telf2'] = temp1['telf2'].str.replace(' ', '')
temp1['telf2'] = temp1['telf2'].str.replace('(', '')
temp1['telf2'] = temp1['telf2'].str.replace(')', '')

In [ ]:
temp1.columns

In [ ]:
temp1[temp1['ctnum'] == '211061046'].kunnr_y.iloc[0]

In [ ]:
tm[tm['kunnr'] == 'c1cbb1245842160c407e61f6587937b4879dcf882a73567bec42052e62a21d39']

In [ ]:
temp1.drop(['ctnum_hash', 'kunnr_hash', 'pck_str', 'b_ymd', 'sex', 'stcd2', 'kunnr_y'], axis=1, inplace=True)

In [ ]:
temp1 = temp1.rename(columns={
      'kunnr_x' : '고객번호',
      'ctnum' : '계약번호',
      'cstat' : '계약상태',
      'gad_name' : '상호',
      'gad_start' : '경비개시일',
      'conn_start' : '계약개시일',
      'conn_end' : '계약종료일',
      'monexp_amt' : '월정료',
      'conn_type' : '계약타입',
      'ctype_l' : '채널(대)',
      'ctype_m' : '채널(중)',
      'ctype_s' : '채널(소)',
      'itype_l' : '업종(대)',
      'itype_m' : '업종(중)',
      'itype_s' : '업종(소)',
      'name1' : '이름1',
      'gad_add01' : '경비물건주소',
      'post_add01' : '우편물수령지주소1',
      'first_bildate' : '최초청구일',
      'telf2' : '전화번호',
      'line_type' : '관제회선구분',
      'uptae' : '업태',
      'jongmok' : '종목',
      'gros_code' : '그로스코드',
      'gross_type' : '그로스타입',
      'kvgr4' : '세그먼트',
      'gad_pstlz' : '우편번호',
      'gad_add02' : '경비물건주소상세',
      'post_add02' : '우편물수령지주소2',
      'sms_yn' : 'SMS동의',
      'email_yn' : 'EMAIL동의',
      'tm_yn' : 'TM동의',
      'dm_yn' : 'DM동의',
      'sk1_yn' : 'SK1동의',
      'sk2_yn' : 'SK2동의',
      'adt1_yn' : '마케팅수신동의',
      'aedat' : '작업일자',
      'aenam' : '변경자',
      'ctrt_id' : 'n사계약번호',
      'talkenddate' : '최종접촉일',
      'contactendtime' : '최종접촉시간',
      'dialresultcd' : '최종접촉결과',
      'contact_cnt' : '접촉회차'
})

In [ ]:
temp1 = temp1[
    ['고객번호', '계약번호', '계약상태', '상호', '경비개시일', '계약개시일', '계약종료일', 
     '월정료', '계약타입', '채널(대)', '채널(중)', '채널(소)', '업종(대)', '업종(중)', '업종(소)', 
           '이름1', '경비물건주소', '우편물수령지주소1', '최초청구일', '전화번호',
           '관제회선구분', '업태', '종목', '그로스코드', '그로스타입', '세그먼트', '우편번호', '경비물건주소상세',
           '우편물수령지주소2', 'SMS동의', 'EMAIL동의', 'TM동의', 'DM동의', 'SK1동의', 'SK2동의', '마케팅수신동의',
           '작업일자', '변경자', 'n사계약번호', '최종접촉일', '최종접촉시간', '최종접촉결과',
           '접촉회차']
]

In [ ]:
temp1.to_csv('/home/skinet/hojin/tm_upsell/N_cms_new_db_201104.csv', encoding='utf-8-sig')

In [ ]:
# magam_copy.drop(['ctnum_hash', 'kunnr_hash', 'pck_str', 'b_ymd', 'sex', 'stcd2', 'ctnum_x', 's25csq'], axis=1, inplace=True)

In [ ]:
# magam_copy = magam_copy.rename(columns={
#       'kunnr' : '고객번호',
#       'ctnum_x' : '계약번호',
#       'cstat' : '계약상태',
#       'gad_name' : '상호',
#       'gad_start' : '경비개시일',
#       'conn_start' : '계약개시일',
#       'conn_end' : '계약종료일',
#       'monexp_amt' : '월정료',
#       'conn_type' : '계약타입',
#       'ctype_l' : '채널(대)',
#       'ctype_m' : '채널(중)',
#       'ctype_s' : '채널(소)',
#       'itype_l' : '업종(대)',
#       'itype_m' : '업종(중)',
#       'itype_s' : '업종(소)',
#       'name1' : '이름1',
#       'gad_add01' : '경비물건주소',
#       'post_add01' : '우편물수령지주소1',
#       'first_bildate' : '최초청구일',
#       'telf2' : '전화번호',
#       'line_type' : '관제회선구분',
#       'uptae' : '업태',
#       'jongmok' : '종목',
#       'gros_code' : '그로스코드',
#       'gross_type' : '그로스타입',
#       'kvgr4' : '세그먼트',
#       'gad_pstlz' : '우편번호',
#       'gad_add02' : '경비물건주소상세',
#       'post_add02' : '우편물수령지주소2',
#       'sms_yn' : 'SMS동의',
#       'email_yn' : 'EMAIL동의',
#       'tm_yn' : 'TM동의',
#       'dm_yn' : 'DM동의',
#       'sk1_yn' : 'SK1동의',
#       'sk2_yn' : 'SK2동의',
#       'adt1_yn' : '마케팅수신동의',
#       'aedat' : '작업일자',
#       'aenam' : '변경자',
#       'ctrt_id' : 'n사계약번호',
#       'rqnum' : '변경요청번호',
#       'ins_type' : '보험구분',
#       'ins_amt' : '보험금액',
#       'ins_amt_type' : '배상액구분',
#       'ins_payment' : '보험납부금액',
#       'ins_stat' : '보험현상태구분',
#       'ins_stat_nm' : '보험현상태구분이름',
#       'ins_amt_type_nm' : '배상액구분이름',
#       'talkenddate' : '최종접촉일',
#       'contactendtime' : '최종접촉시간',
#       'dialresultcd' : '최종접촉결과',
#       'contact_cnt' : '접촉회차'
# })

In [ ]:
# magam_copy.to_csv('/home/skinet/hojin/tm_upsell/9월마감.csv', encoding='utf-8-sig')

In [ ]:
# magam_copy.shape

In [ ]:
hash_table = load_from_athena(conn, '''
SELECT *
  FROM adt_work.hash_info
'''
                             )